In [4]:
import compute_convergence
import data
import glob
import seaborn as sns
import numpy as np
import pandas as pd
import re

sns.set_theme()

# Data of experiments

In [81]:
biased = data.load_data("../output/biased/synthetic/s12/")
hybrid = data.load_data("../output/hybrid/synthetic/s12/")
#biased = data.postprocess(biased)

## Prepare the hybrid dataset

In [230]:
hybrid['lp'] = hybrid.file.str.split('/').str[-1].str.split('_').str[-3] == 'lptrue'
hybrid['tps'] = hybrid.file.str.split('/').str[-1].str.split('_').str[-2].str[3:].astype(int)

In [231]:
cols = [ n for n in hybrid.columns if not re.fullmatch('.*_[0-9]*',n) ]

In [232]:
dta = hybrid.copy()
print(dta.shape)
#dta['ll'] = dta.apply(lambda row: row[[f"llk_{i}" for i in range(300)]].values, axis=1)
dta["pns"]= dta.apply(lambda row: row[[f"pns_{i}" for i in range(300)]].values, axis=1)
dta["iter"]= dta.apply(lambda row: row[[f"iter_{i}" for i in range(300)]].values, axis=1)
dta['pns_l'] = dta.pns.apply(min)
dta['pns_u'] = dta.pns.apply(max)
dta = dta[np.abs(dta.pns_l - dta.pns_u) > 0.02]
outdata_hybrid=dta[['pns', "iter", "pns_l", "pns_u", 'file', "model_type",'lp']]
hybrid.model_type.unique()

(656, 627)


array(['Dobs+Dxb', 'Dobs+Dx+Dxb ', 'Dx+Dxb', 'Dxb', 'Dx'], dtype=object)

# Compute Intevals for different experiments (Hybrid)

Index the file and pivot on the model_type, showing the values of the pns as output, no aggregation is needed as there is only one file-model_type pair

In [264]:
condition = outdata_hybrid.lp
dta = outdata_hybrid[condition].pivot_table(values=["pns_l", "pns_u"], index="file", columns="model_type")
dta.columns = ["_".join(c) for c in dta.columns]
dta.to_csv("hybrid_LPTrue.csv")

condition = ~outdata_hybrid.lp
dta = outdata_hybrid[condition].pivot_table(values=["pns_l", "pns_u"], index="file", columns="model_type")
dta.columns = ["_".join(c) for c in dta.columns]
dta.to_csv("hybrid_LPFalse.csv")


# NAs

In [14]:
pd.DataFrame(pnsu.isna().sum(), columns=["upper"]).join(pd.DataFrame(pnsl.isna().sum(), columns=["lower"]))

,upper,lower
model_type,,
Dobs+Dx+Dxb,7,7
Dobs+Dxb,7,7
Dx+Dxb,5,5
Dxb,1,1


## Count dominating cases

Count the number of experiments where the lower pns of some model_type is >, < or == to another

In [265]:
# count rows > columns
def process(pnsl, pnsu, order):
    pns_data = pnsl
    pnsl_df_gt = pd.DataFrame(
        [ [ sum(pns_data[r] > pns_data[c]) for c in pns_data.columns ] for r in pns_data.columns], 
        columns=pns_data.columns, 
        index=pns_data.columns)

    pns_data = pnsl
    pnsl_df_lt = pd.DataFrame(
        [ [ sum(pns_data[r] < pns_data[c]) for c in pns_data.columns ] for r in pns_data.columns], 
        columns=pns_data.columns, 
        index=pns_data.columns)

    pns_data = pnsl
    pnsl_df_eq = pd.DataFrame(
        [ [ sum(pns_data[r] == pns_data[c]) for c in pns_data.columns ] for r in pns_data.columns], 
        columns=pns_data.columns, 
        index=pns_data.columns)

    pns_data = pnsu
    pnsu_df_gt = pd.DataFrame(
        [ [ sum(pns_data[r] > pns_data[c]) for c in pns_data.columns ] for r in pns_data.columns], 
        columns=pns_data.columns, 
        index=pns_data.columns)

    pns_data = pnsu
    pnsu_df_lt = pd.DataFrame(
        [ [ sum(pns_data[r] < pns_data[c]) for c in pns_data.columns ] for r in pns_data.columns], 
        columns=pns_data.columns, 
        index=pns_data.columns)

    pns_data = pnsu
    pnsu_df_eq = pd.DataFrame(
        [ [ sum(pns_data[r] == pns_data[c]) for c in pns_data.columns ] for r in pns_data.columns], 
        columns=pns_data.columns, 
        index=pns_data.columns)

    # Overall
    cols = pnsu.columns
    containement = pd.DataFrame(
        [ [ sum((pnsl[r] >= pnsl[c]) & (pnsu[r] <= pnsu[c])) for c in cols ] for r in cols], 
        columns=cols, 
        index=cols)

    return containement[order].loc[order]



In [267]:

condition = ~outdata_hybrid.lp
pnsl = outdata_hybrid[condition].pivot_table(values=["pns_l"], index="file", columns="model_type")["pns_l"]
pnsu = outdata_hybrid[condition].pivot_table(values=["pns_u"], index="file", columns="model_type")["pns_u"]
process(pnsl, pnsu, ["Dobs+Dx+Dxb ", "Dobs+Dxb", "Dx+Dxb","Dxb", "Dx"])

model_type,Dobs+Dx+Dxb,Dobs+Dxb,Dx+Dxb,Dxb,Dx
model_type,,,,,
Dobs+Dx+Dxb,40,7,5,21,6
Dobs+Dxb,9,40,6,20,8
Dx+Dxb,0,3,41,25,6
Dxb,1,2,1,45,2
Dx,4,3,13,22,41


In [271]:
condition = outdata_hybrid.lp
pnsl = outdata_hybrid[condition].pivot_table(values=["pns_l"], index="file", columns="model_type")["pns_l"]
pnsu = outdata_hybrid[condition].pivot_table(values=["pns_u"], index="file", columns="model_type")["pns_u"]
process(pnsl, pnsu, ["Dobs+Dx+Dxb ", "Dobs+Dxb", "Dx+Dxb","Dxb"])



model_type,Dobs+Dx+Dxb,Dobs+Dxb,Dx+Dxb,Dxb
model_type,,,,
Dobs+Dx+Dxb,69,15,10,36
Dobs+Dxb,14,69,13,35
Dx+Dxb,12,11,71,40
Dxb,6,4,2,75


# Biased Data


## Prepare the biased dataset

In [152]:
cols = [ n for n in biased.columns if not re.fullmatch('.*_[0-9]*',n) ]


In [153]:
dta = biased.copy()
print(dta.shape)
dta['ll'] = dta.apply(lambda row: row[[f"llk_{i}" for i in range(300)]].values, axis=1)
dta["pns"]= dta.apply(lambda row: row[[f"pns_{i}" for i in range(300)]].values, axis=1)
dta["sizes"]= dta.apply(lambda row: row[[f"trajectory_size_{i}" for i in range(300)]].values, axis=1)

dta = dta[np.abs(dta.pns_l - dta.pns_u) > 0.02]
outdata_biased=dta[np.concatenate([['pns', 'll'], cols])]

(2469, 924)


In [154]:
a = [ ]
for col in biased.columns:
    match = re.fullmatch("pns_([0-9]*)", col)
    if match:
        a.append(int(match[1]))
len(a)

300

In [223]:
reference = outdata_biased[outdata_biased.ps1 == 0].groupby("file").min("index").reset_index()[['file', 'pns_l', 'pns_u']].rename({"pns_l":"astar", "pns_u":"bstar"}, axis=1)


In [224]:
merged = outdata_biased.merge(reference, on='file')

In [228]:
merged['rrmse'] = rrmse(merged.pns_l, merged.pns_u, merged.astar, merged.bstar)

In [205]:
def rrmse(a,b,astar,bstar):
    return np.sqrt((a - astar) ** 2 + (b - bstar) ** 2) / (2 * (bstar - astar))

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

file
../output/biased/synthetic/s12/random_mc2_n10_mid3_d1000_05_mr098_r10_51_mIter500_wtrue_sparents3_x300_0.csv    (-0.0002, 0.0]
../output/biased/synthetic/s12/random_mc2_n10_mid3_d1000_05_mr098_r10_52_mIter500_wtrue_sparents3_x300_0.csv    (-0.0002, 0.0]
../output/biased/synthetic/s12/random_mc2_n10_mid3_d1000_05_mr098_r10_93_mIter500_wtrue_sparents3_x300_0.csv    (-0.0002, 0.0]
../output/biased/synthetic/s12/random_mc2_n11_mid3_d1000_05_mr098_r10_90_mIter500_wtrue_sparents3_x300_0.csv    (-0.0002, 0.0]
../output/biased/synthetic/s12/random_mc2_n5_mid3_d1000_05_mr098_r10_0_mIter500_wtrue_sparents3_x300_0.csv      (-0.0002, 0.0]
                                                                                                                     ...      
../output/biased/synthetic/s12/random_mc2_n9_mid3_d1000_05_mr098_r10_44_mIter500_wtrue_sparents3_x300_0.csv     (-0.0002, 0.0]
../output/biased/synthetic/s12/random_mc2_n9_mid3_d1000_05_mr098_r10_45_mIter500_wtrue_sparents3_x300_0.cs

In [143]:
np.array(biased.columns)

array(['index', 'treewdith', 'bias_endo_treewdith', 'method',
       'endo_treewdith', 'bias_treewdith', 'model_path', 'ps1', 'll_max',
       'ace_l', 'pns_u', 'selector', 'ace_u', 'pns_l', 'pns_91', 'pns_92',
       'pns_90', 'pns_95', 'pns_96', 'pns_93', 'pns_94', 'pns_99',
       'pns_97', 'pns_98', 'pns_80', 'pns_81', 'pns_84', 'pns_85',
       'pns_82', 'pns_83', 'pns_88', 'pns_89', 'pns_86', 'pns_87',
       'pns_70', 'pns_73', 'pns_74', 'pns_71', 'pns_72', 'pns_77',
       'pns_78', 'pns_75', 'pns_76', 'pns_79', 'pns_62', 'pns_63',
       'pns_60', 'pns_61', 'pns_66', 'pns_67', 'pns_64', 'pns_65',
       'pns_68', 'pns_69', 'pns_199', 'pns_198', 'pns_197', 'pns_196',
       'pns_195', 'pns_194', 'pns_193', 'pns_192', 'pns_11', 'pns_12',
       'pns_10', 'pns_15', 'pns_16', 'pns_13', 'pns_14', 'pns_19',
       'pns_17', 'pns_18', 'time_learn', 'trajectory_size_7',
       'trajectory_size_6', 'trajectory_size_9', 'trajectory_size_8',
       'trajectory_size_1', 'trajectory_size_0

In [ ]:
outdata_biased.to_json("pns_12_biased.json")

In [58]:
ps1 = biased[(biased.method == "EMCC") & (biased.ps1 == 1)]

In [66]:
simple = [ c for c in ps1.columns if not ( c.startswith("llk_") | c.startswith("ratio_") | c.startswith("trajectory_") | c.startswith("pns_")) ]

In [124]:
biased[(biased.method == "EMCC")][simple]

,index,treewdith,bias_endo_treewdith,method,endo_treewdith,bias_treewdith,model_path,ps1,ll_max,ace_l,...,time_learn,time_pns,sparents,file,network_type,mc,n,mid,d,inf
1,1,3.0,NaN,EMCC,3.0,NaN,/mnt/beegfs/rcabanas/dev/credici/papers/journa...,1.000,-1981.064670,NaN,...,152006.0,254.0,NaN,../output/biased/synthetic/s12/random_mc2_n5_m...,random,2,5,3,1000,wtrue
2,2,3.0,4.0,EMCC,3.0,4.0,/mnt/beegfs/rcabanas/dev/credici/papers/journa...,0.000,-1981.064670,NaN,...,82446.0,115.0,[1|3|4],../output/biased/synthetic/s12/random_mc2_n5_m...,random,2,5,3,1000,wtrue
3,3,3.0,4.0,EMCC,3.0,4.0,/mnt/beegfs/rcabanas/dev/credici/papers/journa...,0.046,-1981.064670,NaN,...,129639.0,72.0,[1|3|4],../output/biased/synthetic/s12/random_mc2_n5_m...,random,2,5,3,1000,wtrue
4,4,3.0,4.0,EMCC,3.0,4.0,/mnt/beegfs/rcabanas/dev/credici/papers/journa...,0.193,-1981.064670,NaN,...,185201.0,67.0,[1|3|4],../output/biased/synthetic/s12/random_mc2_n5_m...,random,2,5,3,1000,wtrue
5,5,3.0,4.0,EMCC,3.0,4.0,/mnt/beegfs/rcabanas/dev/credici/papers/journa...,0.394,-1981.064670,NaN,...,229875.0,66.0,[1|3|4],../output/biased/synthetic/s12/random_mc2_n5_m...,random,2,5,3,1000,wtrue
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2464,6,3.0,4.0,EMCC,3.0,4.0,/mnt/beegfs/rcabanas/dev/credici/papers/journa...,0.450,-1779.330943,NaN,...,606710.0,252.0,[0|3|4],../output/biased/synthetic/s12/random_mc2_n6_m...,random,2,6,3,1000,wtrue
2465,7,3.0,4.0,EMCC,3.0,4.0,/mnt/beegfs/rcabanas/dev/credici/papers/journa...,0.653,-1779.330943,NaN,...,459788.0,229.0,[0|3|4],../output/biased/synthetic/s12/random_mc2_n6_m...,random,2,6,3,1000,wtrue
2466,8,3.0,4.0,EMCC,3.0,4.0,/mnt/beegfs/rcabanas/dev/credici/papers/journa...,0.690,-1779.330943,NaN,...,494388.0,278.0,[0|3|4],../output/biased/synthetic/s12/random_mc2_n6_m...,random,2,6,3,1000,wtrue
2467,9,3.0,4.0,EMCC,3.0,4.0,/mnt/beegfs/rcabanas/dev/credici/papers/journa...,0.975,-1779.330943,NaN,...,544941.0,129.0,[0|3|4],../output/biased/synthetic/s12/random_mc2_n6_m...,random,2,6,3,1000,wtrue
